<a href="https://colab.research.google.com/github/Rion-user/Welcome/blob/master/new_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 설치
!pip install xmltodict
!pip install load_dotenv
!pip install requests
!pip install tqdm

# 라이브러리 호출
import requests
import xmltodict
import json
import os
import pandas as pd
from tqdm import tqdm

OC = 'nellcome4636'
os.chdir('/content/drive/MyDrive/KB')

In [ ]:
# 데이터 개수 en: 2586 kr: 5273
# 예상페이지 수: 26page (2586 // 100)
# 예상페이지 수: 53page (5273 // 100)

# MST(일련번호) 모두 가져오기
en_id = set()
kr_id = set()

for page in tqdm(range(1,30)):
    # print(page, "/ 26")
    list_url = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target=elaw&page={page}&display=100'
    response = requests.get(list_url)
    json_data = xmltodict.parse(response.text)
    if 'law' not in json_data['LawSearch'] or len(json_data['LawSearch']['law']) == 0:
        break
    for law in json_data['LawSearch']['law']:
      en_id.add(law['법령ID'])

for page in tqdm(range(1,60)):
    # print(page, "/ 53")
    list_url = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target=law&page={page}&display=100'
    response = requests.get(list_url)
    json_data = xmltodict.parse(response.text)
    if 'law' not in json_data['LawSearch'] or len(json_data['LawSearch']['law']) == 0:
        break
    for law in json_data['LawSearch']['law']:
        kr_id.add(law['법령ID'])

 90%|████████▉ | 53/59 [00:45<00:05,  1.18it/s]


In [ ]:
# 2348
from tqdm import tqdm

same_jo_mst = []
diff_jo_mst = []
err_mst = []

# 법령별 정보 추출
for id in tqdm(en_id):
    try:
        # en
        url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&ID={id}'
        response_en = requests.get(url)
        if response_en.status_code != 200:
            err_mst.append((id, response_en.status_code, 'en'))
            continue

        # kr
        url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&ID={id}'
        response_kr = requests.get(url)
        if response_kr.status_code != 200:
            err_mst.append((id, response_kr.status_code, 'kr'))
            continue

        # en, kr parsing
        json_data_en = xmltodict.parse(response_en.text)
        json_data_kr = xmltodict.parse(response_kr.text)

        # check 1
        # 조문개수 확인
        if len(json_data_en['Law']['JoSection']['Jo']) == len(json_data_kr['법령']['조문']['조문단위']):
            same_jo_mst.append(id)
        else:
            diff_jo_mst.append(id)
    except:
        err_mst.append((id, 'unknown','need to check'))

100%|██████████| 2587/2587 [2:56:29<00:00,  4.09s/it]


In [ ]:
print("same")
print(same_jo_mst)
print("diff")
print(diff_jo_mst)
print("err")
print(err_mst)

same
['013781', '000179', '011556', '000815', '009543', '010359', '000014', '011150', '011350', '010583', '011554', '010937', '014017', '011459', '005674', '012269', '000325', '000009', '013783', '010168', '001967', '011774', '012169', '009848', '007778', '003637', '009480', '010724', '001220', '000193', '012658', '011537', '010923', '000054', '009765', '012442', '009692', '013671', '001814', '011352', '006871', '011173', '009627', '001144', '011008', '012633', '012309', '009643', '000015', '003783', '000062', '005354', '003835', '005214', '001962', '012320', '004119', '001221', '001282', '003779', '000317', '010011', '000478', '004004', '001466', '012105', '009503', '009884', '011176', '002260', '011503', '013756', '010097', '011154', '001242', '009323', '013758', '012020', '003717', '013030', '002009', '013719', '009637', '011431', '001136', '001134', '009353', '005030', '010164', '009609', '009674', '000592', '011525', '011585', '000036', '010987', '002679', '012678', '002199', '001

In [ ]:
same_jo_mst = ['013781', '000179', '011556', '000815', '009543', '010359', '000014', '011150', '011350', '010583', '011554', '010937', '014017', '011459', '005674', '012269', '000325', '000009', '013783', '010168', '001967', '011774', '012169', '009848', '007778', '003637', '009480', '010724', '001220', '000193', '012658', '011537', '010923', '000054', '009765', '012442', '009692', '013671', '001814', '011352', '006871', '011173', '009627', '001144', '011008', '012633', '012309', '009643', '000015', '003783', '000062', '005354', '003835', '005214', '001962', '012320', '004119', '001221', '001282', '003779', '000317', '010011', '000478', '004004', '001466', '012105', '009503', '009884', '011176', '002260', '011503', '013756', '010097', '011154', '001242', '009323', '013758', '012020', '003717', '013030', '002009', '013719', '009637', '011431', '001136', '001134', '009353', '005030', '010164', '009609', '009674', '000592', '011525', '011585', '000036', '010987', '002679', '012678', '002199', '001644', '003610', '000890', '002378', '012322', '001765', '012872', '000092', '009368', '001807', '009996', '011022', '001823', '010595', '001983', '001472', '001327', '001978', '009999', '012486', '010758', '013147', '003027', '011305', '003492', '009811', '000867', '001226', '010180', '011557', '010458', '000088', '000762', '011552', '009536', '012843', '010878', '005452', '001247', '013140', '003147', '002619', '010358', '012569', '000002', '005273', '012017', '000184', '005651', '014002', '000399', '009399', '012696', '011329', '003997', '013126', '010356', '001119', '001761', '002947', '001648', '011315', '001545', '003435', '009707', '003788', '003361', '000753', '014202', '001291', '013894', '002096', '001412', '000865', '004379', '010716', '002035', '000776', '010597', '000041', '001205', '012483', '009844', '000888', '012870', '002002', '013639', '011349', '010710', '000784', '014213', '012557', '012291', '003297', '000239', '011531', '004043', '011994', '012863', '000944', '010297', '000822', '014099', '012521', '000319', '001432', '003684', '004242', '009256', '010403', '010451', '010761', '010722', '010835', '002649', '001712', '012313', '009764', '011319', '004057', '013479', '001737', '012578', '012338', '002643', '011435', '002028', '012071', '000755', '011167', '011031', '001836', '003000', '012481', '000975', '001785', '011020', '001849', '012484', '010362', '013041', '000380', '002221', '005220', '001751', '002383', '010375', '000035', '012266', '001989', '004262', '010490', '010728', '012045', '001669', '001465', '004498', '009401', '000565', '001425', '012026', '001790', '003655', '011365', '011136', '009518', '001865', '001118', '001674', '007196', '010184', '011513', '001867', '009290', '002228', '010511', '011463', '002006', '012508', '011367', '001965', '005361', '009907', '011408', '001263', '010411', '010360', '000641', '000204', '011545', '010512', '001233', '011384', '009403', '004605', '002546', '000135', '004526', '011366', '012003', '007717', '003122', '003659', '011438', '012314', '012228', '001839', '000324', '003424', '012564', '012312', '000981', '002303', '000249', '001284', '011149', '010235', '011373', '010179', '012051', '009624', '000096', '004771', '005370', '009346', '012095', '001627', '009629', '011151', '005676', '010461', '012522', '003989', '011360', '001206', '002038', '002034', '001863', '000028', '010830', '002262', '005668', '010635', '013702', '012746', '001360', '010436', '005318', '012676', '010725', '001662', '009258', '012334', '010166', '009581', '013459', '012585', '002000', '010378', '001428', '001123', '002937', '003086', '011621', '000520', '012556', '000141', '009513', '001459', '005595', '011712', '003055', '010957', '011017', '002111', '013617', '004822', '000001', '010910', '003780', '012373', '011096', '009511', '011009', '011423', '001371', '012001', '009269', '010944', '002144', '005578', '001404', '012145', '011540', '000892', '009845', '004662', '005326', '010681', '000549', '012246', '013703', '000685', '004109', '010714', '012159', '013688', '010202', '005247', '013453', '010589', '010786', '001794', '004610', '001861', '012498', '000448', '012440', '013012', '011546', '000240', '012499', '012799', '000064', '009586', '010432', '012571', '010530', '013021', '012331', '005156', '000012', '012183', '011181', '012103', '001292', '001809', '013099', '000086', '001410', '004034', '009885', '011941', '000409', '012235', '011442', '000928', '004214', '009189', '000219', '012151', '001180', '011169', '001735', '009874', '012463', '002255', '009684', '009474', '010908', '012197', '002653', '012261', '001241', '000168', '004037', '010729', '009621', '010062', '011494', '009225', '002655', '011523', '001626', '000623', '008556', '011175', '010382', '000502', '003904', '009877', '003126', '010188', '012401', '009883', '011432', '010893', '000667', '000730', '013464', '001000', '010594', '009721', '010071', '011995', '012193', '011330', '001027', '011495', '010088', '001365', '012492', '010000', '002388', '010291', '005530', '003683', '001135', '000754', '009653', '004481', '011235', '001430', '009632', '012248', '012566', '002982', '011215', '010946', '011732', '003004', '011826', '000008', '011062', '000416', '012027', '000004', '009283', '002109', '014182', '005401', '003391', '003205', '013104', '012416', '009747', '000182', '010779', '001710', '009145', '011140', '002373', '000122', '000318', '000906', '011990', '004385', '000758', '011188', '007643', '013682', '003124', '011551', '001100', '001818', '010491', '007188', '001587', '009736', '013647', '012239', '009881', '012024', '012804', '013138', '011018', '000445', '013050', '012262', '012189', '001219', '011679', '001182', '002351', '013091', '014081', '011170', '012636', '012580', '014130', '000047', '012242', '011006', '011917', '000946', '010381', '000178', '007397', '000747', '000864', '009588', '001020', '004949', '000083', '010685', '001736', '000506', '004270', '001780', '001434', '002031', '000254', '011563', '013744', '012106', '009479', '009475', '009221', '012227', '003531', '012225', '004100', '000813', '001624', '010598', '009280', '009338', '012893', '011211', '002931', '000216', '001435', '014312', '003136', '013569', '005233', '010711', '013663', '011363', '002847', '004482', '009380', '000994', '009934', '012220', '000297', '001261', '010236', '011187', '013098', '011566', '003161', '013668', '010842', '005647', '011179', '010361', '012514', '010420', '001215', '012561', '001293', '005145', '012069', '009349', '000284', '012512', '013615', '012268', '005603', '001676', '010538', '003621', '002711', '000243', '014049', '000868', '011897', '009271', '000241', '010007', '001595', '012308', '011893', '004766', '013718', '013755', '011929', '014151', '001438', '013779', '011869', '009642', '012468', '001988', '010510', '011199', '012632', '012882', '008585', '005042', '004641', '011486', '012709', '000507', '001170', '000080', '001852', '013686', '001469', '011061', '004487', '011184', '012769', '001850', '009924', '002039', '001258', '012295', '011895', '001721', '012297', '009926', '010783', '009933', '010528', '009365', '001179', '009377', '009810', '000066', '011878', '012146', '010308', '002040', '000221', '010012', '000710', '009557', '004002', '012631', '010590', '000912', '013201', '003809', '011234', '009135', '011418', '009281', '011019', '000398', '003979', '001376', '010915', '012424', '014112', '011189', '000941', '009644', '001462', '001851', '011473', '002049', '001155', '012567', '000760', '010198', '010111', '001593', '009626', '010727', '004040', '001615', '001746', '011998', '009156', '010833', '011619', '013167', '012573', '009270', '004608', '001228', '010365', '010038', '013685', '001660', '003166', '014160', '002389', '001218', '000407', '000248', '001677', '002813', '011475', '004097', '010586', '012049', '012068', '001668', '001718', '000328', '005630', '001427', '000564', '000749', '001461', '009876', '009761', '011390', '009677', '009932', '002118', '001406', '013614', '001246', '003116', '000027', '011868', '001979', '005600', '001113', '000923', '004226', '010463', '012825', '012067', '000851', '000591', '001424', '000980', '012032', '010053', '011439', '005615', '007032', '000190', '003070', '000783', '000759', '001281', '010913', '000947', '008234', '009426', '009529', '001692', '001589', '012293', '003572', '001445', '012568', '012842', '003786', '000999', '002033', '012321', '002089', '001372', '010447', '009836', '000690', '009170', '004960', '001014', '005687', '000966', '000996', '011723', '013687', '009295', '012892', '000837', '013626', '001159', '005336', '012198', '013476', '014201', '010176', '011527', '001454', '011614', '013572', '003672', '011682', '014214', '010217', '010486', '001006', '013566', '011632', '009672', '003476', '000857', '013210', '000901', '012264', '011879', '010587', '011601', '014152', '011420', '004958', '013735', '012245', '010540', '001709', '010485', '011409', '013761', '011196', '012418', '000446', '000853', '003118', '009485', '010479', '002018', '003058', '010708', '004217', '009192', '009657', '013610', '011203', '010306', '011892', '000880', '001021', '004171', '011758', '012018', '010187', '000285', '013741', '001985', '011321', '011370', '001147', '010914', '005397', '002136', '001132', '004291', '000895', '012123', '001197', '004974', '013246', '011714', '011574', '001202', '010956', '011141', '003489', '001230', '013623', '003916', '000943', '012507', '001730', '000899', '001513', '009625', '001109', '000579', '000533', '010200', '012712', '013235', '009846', '011541', '012052', '013193', '010435', '009925', '010718', '003311', '012797', '004698', '002015', '012574', '011362', '001719', '001998', '011202', '010523', '003723', '012700', '000920', '009940', '001097', '001467', '012196', '012818', '000848', '010449', '002115', '005396', '005408', '000929', '009199', '013967', '003519', '005157', '010410', '010191', '012701', '001244', '012413', '011986', '011236', '001504', '003107', '001596', '003706', '000370', '013669', '004628', '009915', '000898', '011444', '001133', '003421', '010947', '011903', '011021', '000855', '001697', '005688', '000408', '012969', '003313', '011505', '011434', '012311', '009358', '010503', '001238', '003986', '000921', '012559', '004198', '002010', '012329', '000887', '002108', '010052', '001629', '000854', '010941', '009758', '011220', '001827', '001548', '011539', '009318', '010229', '001143', '003121', '001343', '009218', '010843', '010002', '010608', '000936', '011778', '013136', '003650', '011168', '002646', '012199', '005377', '013470', '003143', '001326', '013608', '013767', '011361', '001477', '000850', '000531', '000953', '005340', '001173', '010864', '010943', '010876', '002011', '009195', '012260', '000191', '010209', '010942', '000577', '014046', '010307', '011377', '000832', '012287', '000007', '009409', '001872', '011092', '001283', '009908', '013069', '005212', '000006', '014255', '010366', '009712', '010443', '009137', '000061', '011078', '010965', '011620', '010082', '009608', '013219', '000139', '001973', '011250', '001125', '010373', '001755', '011374', '001264', '012620', '013774', '000385', '001213', '010612', '005353', '005551', '001458', '003462', '010721', '011351', '010001', '014082', '001631', '000751', '000235', '013135', '012707', '001408', '012002', '013115', '005140', '009638', '002029', '009936', '001112', '001214', '013103', '001142', '000346', '003618', '013785', '000984', '010539', '010379', '001732', '000433', '001256', '012788', '010911', '012066', '000127', '000705', '013481', '013468', '002020', '009372', '004166', '000394', '010502', '000123', '000207', '000728', '011536', '013034', '014094', '009181', '001866', '010561', '003931', '013113', '001499', '010493', '013792', '011371', '001963', '002127', '001015', '001129', '002043', '003325', '004223', '011058', '005348', '001116', '014062', '011559', '002254', '001871', '010488', '011562', '009682', '012315', '003748', '012661', '003666', '002005', '003907', '013220', '011999', '001729', '005159', '010668', '000696', '011529', '001433', '012154', '010222', '012107', '011857', '005343', '012593', '003591', '013780', '012142', '010472', '013180', '011567', '012111', '007270', '011358', '001853', '001744', '012317', '010600', '000463', '001399', '002933', '001126', '011474', '001997', '009681', '000646', '010726', '009634', '004798', '011928', '004344', '011142', '013889', '014127', '001217', '004940', '010667', '004963', '011139', '011532', '006350', '000171', '005057', '009656', '002016', '002404', '005666', '013993', '003948', '013019', '000426', '004136', '004692', '002521', '003695', '012181', '009396', '001177', '011953', '002801', '011997', '013789', '013705', '004530', '001583', '011209', '011307', '009534', '013847', '001525', '005618', '001347', '010591', '005677', '005383', '001628', '012747', '010602', '004231', '000236', '009942', '001486', '001771', '010277', '000242', '004066', '012073', '002936', '010756', '010900', '009947', '009998', '014071', '000134', '010770', '001999', '001507', '009522', '004950', '009163', '000283', '001603', '005146', '012060', '001286', '009910', '000299', '001854', '004174', '000474', '010109', '006326', '010844', '000852', '011359', '003057', '004704', '000266', '007575', '000583', '009371', '000836', '000935', '010886', '010294', '001974', '010593', '004132', '001763', '002008', '002681', '012347', '011057', '010300', '011285', '000023', '012070', '013221', '013245', '011519', '000237', '012670', '012419', '010613', '010123', '012409', '012810', '012494', '003802', '010404', '000169', '011437', '005246', '003778', '007475', '001479', '011153', '001270', '001156', '002537', '000129', '013736', '002246', '001821', '011391', '010534', '011335', '001509', '003751', '011436', '004049', '001837', '012496', '008748', '011267', '001855', '010834', '003400', '001713', '000748', '010484', '000519', '013092', '011152', '001616', '000874', '011564', '003158', '013759', '002146', '009252', '011561', '012592', '009757', '000202', '000419', '012237', '011091', '011107', '009595', '003470', '001550', '013869', '004178', '000925', '009741', '000934', '014063', '010073', '001350', '005223', '004293', '012464', '000049', '010026', '001114', '000977', '003477', '010607', '000886', '000699', '013679', '000188', '002143', '000536', '001993', '012665', '013716', '004470', '000767', '003823', '011406', '001437', '001514', '002413', '001987', '009540', '010872', '003596', '003204', '012412', '011989', '009395', '010008', '010357', '014092', '001457', '003212', '004601', '012491', '000595', '009413', '013631', '011023', '000312', '009857', '005585', '005365', '010437', '003436', '012869', '000889', '001300', '009618', '014295', '001184', '012272', '012267', '011407', '013480', '001964', '000894', '010755', '010865', '001754', '011427', '000979', '003552', '010945', '011348', '011656', '010596', '000170', '003440', '005652', '009542', '001546', '001294', '001777', '000891', '014238', '011143', '013137', '013612', '000003', '013879', '002026', '009997', '013242', '001476', '011144', '013994', '014159', '003632', '009351', '003873', '009265', '011894', '001483', '011570', '000298', '013800', '002053', '002037', '010090', '000964', '000998', '000383', '001723', '013244', '009294', '000752', '000709', '004186', '001460', '002391', '004856', '009174', '005381', '009679', '013662', '013156', '013139', '011825', '011429', '013097', '010452', '010035', '001455', '004680', '005621', '012265', '013114', '011233', '001601', '005028', '010794', '011553', '001537', '001470', '010771', '011339', '000068', '001642', '001148', '003784', '013159', '010112', '013070', '010723', '000121', '004508', '005131', '010759', '009517', '001604', '011067', '010086', '010087', '001178', '000422', '013447', '012480', '010526', '013202', '013031', '010163', '012053', '001257', '012238', '010592', '002853', '012310', '009760', '009255', '013642', '010707', '009837', '009620', '009451', '003461', '010409', '000698', '000621', '003540', '009636', '012050', '013170', '010159', '004705', '000183', '000658', '014217', '001146', '001262', '012292', '010040', '000156', '010085', '010021', '000011', '001212', '001860', '012974', '005137', '009161', '003293', '001194', '009245', '000743', '011186', '010413', '010717', '001791', '005526', '002058', '012497', '005209', '003210', '002317', '012236', '010930', '010201', '014166', '001711', '001185', '012019', '012679', '011548', '004104', '001977', '001359', '000665', '002286', '005249', '011185', '000942', '010055', '010632', '014371', '012443', '001966', '001722']
diff =['010234', '010175', '010228', '009698', '010383', '011887', '004972', '001482', '009500', '003083', '007646', '009361', '012584', '000132', '002248', '010431', '009988', '000161', '010178', '012339', '001670', '009419', '010624', '009696', '002366', '003759', '000162', '003825', '008561', '000900', '010044', '010162', '012427', '002088', '005498', '009943', '002421', '013791', '012524', '001781', '012697', '002599', '011372', '011882', '004590', '002233', '000816', '009486', '007845', '013171', '005518', '001363', '010161', '001638', '000155', '009705', '004758', '011558', '001141', '004830', '010194', '003581', '014070', '011549', '002907', '003727', '005379', '012177', '002322', '000154', '005444', '012296', '002250', '006888', '000289', '012065', '011460', '001870', '005453', '001969', '009592', '004108', '009766', '011340', '007639', '009686', '009589', '011612', '002308', '009930', '012286', '011314', '000117', '011357', '001431', '001720', '011724', '005256', '010213', '011172', '003077', '011675', '006295', '009746', '012247', '010513', '003438', '009948', '000729', '001654', '013056', '005367', '009640', '011177', '012400', '006175', '001643', '013830', '001702', '011013', '001598', '001463', '002820', '010793', '004388', '009264', '001566', '003429', '011491', '013463', '009204', '011996', '011908', '005570', '011881', '013763', '009591', '009740', '000308', '010753', '010199', '000167', '001401', '010267', '009655', '002041', '003172', '009412', '009155', '001337', '010840', '012110', '004862', '004678', '001523', '003922', '000130', '000479', '001248', '005459', '009286', '014041', '010004', '014098', '003395', '001645', '011860', '001784', '010520', '012789', '001416', '010519', '010648', '009410', '000058', '002048', '001607', '001532', '001733', '001706', '009411', '003956', '004626', '011484', '009182', '001671', '010298', '000160', '007642', '000660', '003302', '005352', '010118', '011007', '000552', '002814', '003563', '013807', '010013', '010240', '009897', '005407', '005077', '010916', '010024', '010782', '003613', '011443', '000959', '002044', '000187', '001805', '000016', '000985', '009835', '004977', '002583', '005059', '001549', '003673', '001565', '005669', '000323', '012984', '000185', '010745', '000262', '010704', '009941', '007770', '001742', '004376', '001471', '001501', '000288', '001124', '010005', '001585', '000212', '001740', '009886', '012834', '011306', '000593', '012054', '000824', '013622', '000223', '010414', '009954', '009177', '005098', '003945', '010508', '009863', '010446', '010265', '010909', '006859', '004754', '009244', '010820', '004946', '011492', '012879', '013458', '009205', '001444', '009494', '000956', '013828', '010231', '001538', '005245', '004135', '009369', '002884', '010126', '012201', '007519', '000772', '004936', '003980', '008218', '012500', '001655', '011602', '009878', '001813', '001747', '004797', '013784', '009514', '013094', '001815', '005369', '000798', '003750', '009683', '001829', '011942', '014170', '009628', '003602', '003811', '012437', '001325', '012900', '002025', '001357', '011178', '009847', '001980', '001561', '005329', '001701', '002293', '010273', '004273', '001707', '001971', '001613', '013672', '011468', '001717', '001267', '003072', '003654', '012523', '010623', '011858', '009367', '012360', '009505', '004750', '009356', '007230', '001571', '001748', '005685', '010776', '010873', '001570', '010089', '000091', '009597', '001725', '003556', '000454', '010380', '003617', '009842', '011880', '000166', '004387', '012642', '010402', '001653', '009892', '000830', '010499', '008324', '001700', '004105', '004177', '000165', '012502', '005634', '013574', '009282', '004179', '012513', '001619', '011902', '000695', '004390', '003282', '001236', '009201', '009694', '001846', '000128', '004065', '012637', '010772', '003100', '001362', '009208', '000773', '010712', '012226', '011992', '012438', '012490', '000958', '001328', '003103', '012254', '001766', '009631', '010311', '004243', '000355', '012903', '010588', '009580', '001981', '001699', '012805', '002324', '000133', '000030', '009938', '004138', '004607', '009641', '010430', '001171', '009882', '000173', '002384', '005238', '011147', '009502', '009147', '006590', '000534', '010444', '014320', '007395', '014189', '010846', '010110', '012345', '010158', '003940', '010196', '004552', '009939', '003814', '011709', '007656', '007282', '000669', '000120', '000603', '001377', '000744', '012644', '002541', '010301', '005288', '010709', '009206', '008743', '002249', '001783', '005092', '001760', '001563', '004551', '001539', '010107', '013616', '012551', '011063', '010531', '013643', '000412', '003626', '001344', '003227', '012205', '000222', '003501', '001968', '001708', '010974', '003725', '003513', '013932', '003777', '002567', '009146', '009450', '000143', '010445', '010599', '012495', '010938', '000765', '001774', '012708', '001749', '010817', '009293', '013231', '003793', '000354', '013670', '012046', '009374', '011615', '005308', '000992', '011163', '001590', '011155', '006780', '011896', '001792', '012108', '011313', '014156', '012441', '012008', '000794', '010078', '009873', '009350', '010958', '002237', '010066', '000311', '012318', '004623', '004708', '010400', '013749', '001568', '011522', '009891', '012690', '004240', '002367', '012414', '012878', '004480', '002652', '001734', '012270', '009862', '011718', '011016', '009708', '001378', '009648', '011276', '000820', '009190', '011433', '001592', '001994', '010504', '002833', '002019', '000218', '010730', '011528', '005323', '001591', '012417', '001622', '002024', '002922', '012323', '000885', '011249', '010415', '005206', '012518', '002027', '000074', '012147', '014047', '013203', '001739', '001786', '011633', '002942', '001540', '009276', '003069', '004026', '010165', '003506', '011448', '009136', '001275', '009838', '012517', '010302', '010647', '012203', '011662', '009872', '001623', '001743', '001773', '003754', '001025', '002891', '001618', '007473', '000846', '009521', '002298', '010374', '004216', '013232', '009348', '008741', '011912', '002671', '003140', '011452', '009671', '004342', '013794', '012703', '011113', '010483', '004378']
err = [('004920', 'unknown', 'need to check'), ('002021', 500, 'en'), ('011864', 500, 'en'), ('001201', 500, 'en'), ('009388', 500, 'en'), ('008299', 500, 'en'), ('001843', 500, 'en'), ('004632', 500, 'en'), ('000878', 500, 'en'), ('009593', 500, 'en'), ('010998', 'unknown', 'need to check'), ('000796', 500, 'en'), ('000095', 500, 'en'), ('003623', 'unknown', 'need to check'), ('001728', 'unknown', 'need to check'), ('003586', 500, 'en'), ('001002', 500, 'en'), ('001004', 500, 'en'), ('010553', 500, 'en'), ('005988', 'unknown', 'need to check'), ('009964', 500, 'en'), ('001834', 500, 'en'), ('001842', 500, 'en'), ('000993', 'unknown', 'need to check'), ('001859', 'unknown', 'need to check'), ('003422', 500, 'en'), ('001605', 'unknown', 'need to check'), ('009767', 'unknown', 'need to check'), ('010344', 500, 'en'), ('001361', 500, 'en'), ('001995', 500, 'en'), ('000010', 500, 'en'), ('009833', 500, 'en'), ('000232', 500, 'en'), ('000835', 500, 'en'), ('010558', 500, 'en'), ('004967', 500, 'en'), ('000344', 500, 'en'), ('005292', 'unknown', 'need to check'), ('001599', 500, 'en'), ('000719', 500, 'en'), ('000360', 'unknown', 'need to check'), ('000199', 500, 'en'), ('000954', 500, 'en'), ('013650', 'unknown', 'need to check'), ('000071', 500, 'en'), ('003608', 'unknown', 'need to check'), ('004675', 500, 'en'), ('002315', 'unknown', 'need to check'), ('000126', 500, 'en'), ('005321', 'unknown', 'need to check'), ('000063', 500, 'en'), ('001564', 500, 'en'), ('008586', 500, 'en'), ('004380', 500, 'en'), ('004256', 500, 'en'), ('004017', 500, 'en'), ('001788', 'unknown', 'need to check'), ('013179', 'unknown', 'need to check'), ('009289', 'unknown', 'need to check'), ('000763', 500, 'en'), ('004823', 500, 'en'), ('010197', 'unknown', 'need to check'), ('003084', 500, 'en'), ('001762', 500, 'en'), ('004110', 500, 'en'), ('010500', 500, 'en'), ('009288', 500, 'en'), ('001826', 500, 'en'), ('010822', 'unknown', 'need to check'), ('011388', 500, 'en'), ('001656', 'unknown', 'need to check'), ('002374', 500, 'en'), ('001992', 500, 'en'), ('001695', 500, 'en'), ('013704', 'unknown', 'need to check'), ('003517', 'unknown', 'need to check'), ('005026', 500, 'en'), ('001782', 500, 'en'), ('009603', 500, 'en'), ('010501', 500, 'en'), ('009661', 500, 'en'), ('000800', 500, 'en'), ('001252', 500, 'en'), ('001541', 500, 'en'), ('010434', 'unknown', 'need to check'), ('004713', 500, 'en'), ('011720', 500, 'en'), ('004229', 500, 'en'), ('009291', 500, 'en'), ('011471', 500, 'en'), ('004389', 500, 'en'), ('004484', 500, 'en'), ('000209', 500, 'en'), ('011085', 500, 'en'), ('010713', 'unknown', 'need to check'), ('000793', 500, 'en'), ('006277', 500, 'en'), ('009590', 500, 'en'), ('001795', 500, 'en'), ('004952', 500, 'en'), ('014169', 500, 'en'), ('005364', 500, 'en'), ('004472', 500, 'en'), ('004386', 500, 'en'), ('001529', 500, 'en'), ('010773', 500, 'en'), ('009183', 500, 'en'), ('010731', 500, 'en'), ('000198', 500, 'en'), ('000358', 500, 'en'), ('009317', 500, 'en'), ('011555', 500, 'en'), ('001819', 500, 'en'), ('003112', 500, 'en'), ('004473', 500, 'en'), ('011171', 'unknown', 'need to check'), ('000518', 500, 'en'), ('003524', 500, 'en'), ('001358', 500, 'en'), ('004253', 500, 'en'), ('001745', 500, 'en'), ('000927', 500, 'en'), ('001588', 500, 'en'), ('005259', 500, 'en'), ('009680', 'unknown', 'need to check'), ('009227', 500, 'en'), ('001672', 500, 'en'), ('000516', 500, 'en'), ('002846', 500, 'en'), ('002956', 500, 'en'), ('001515', 500, 'en'), ('000802', 500, 'en'), ('004239', 500, 'en'), ('008591', 500, 'en'), ('000159', 500, 'en'), ('005581', 500, 'en'), ('001195', 'unknown', 'need to check'), ('000750', 500, 'en'), ('000439', 500, 'en'), ('000483', 500, 'en'), ('002003', 500, 'en'), ('001811', 500, 'en'), ('001812', 500, 'en'), ('003693', 'unknown', 'need to check'), ('000353', 500, 'en'), ('011980', 500, 'en'), ('001464', 500, 'en'), ('007880', 'unknown', 'need to check'), ('001584', 'unknown', 'need to check'), ('001753', 500, 'en'), ('001556', 'unknown', 'need to check'), ('001652', 'unknown', 'need to check'), ('006275', 500, 'en'), ('000268', 500, 'en'), ('000995', 500, 'en'), ('001641', 500, 'en'), ('000208', 500, 'en'), ('003009', 500, 'en'), ('012515', 500, 'en'), ('010732', 500, 'en'), ('000675', 500, 'en'), ('009604', 500, 'en'), ('004267', 500, 'en'), ('002400', 500, 'en'), ('010610', 500, 'en'), ('000926', 500, 'en'), ('009587', 500, 'en'), ('001975', 500, 'en'), ('005269', 500, 'en'), ('000176', 500, 'en'), ('000057', 500, 'en'), ('000918', 'unknown', 'need to check'), ('009238', 500, 'en'), ('009726', 500, 'en'), ('001602', 500, 'en'), ('010966', 500, 'en'), ('011441', 500, 'en'), ('001772', 500, 'en'), ('009559', 500, 'en'), ('001199', 500, 'en'), ('011290', 500, 'en'), ('010428', 500, 'en'), ('000791', 500, 'en'), ('004603', 500, 'en'), ('011101', 'unknown', 'need to check'), ('009619', 500, 'en'), ('005512', 500, 'en'), ('011681', 'unknown', 'need to check'), ('004269', 500, 'en'), ('001756', 500, 'en'), ('002110', 500, 'en'), ('004931', 500, 'en'), ('000761', 500, 'en'), ('000539', 500, 'en'), ('000924', 500, 'en'), ('009879', 500, 'en'), ('005134', 500, 'en'), ('003190', 500, 'en'), ('010629', 500, 'en'), ('009690', 500, 'en'), ('005539', 500, 'en'), ('001533', 500, 'en'), ('001535', 500, 'en'), ('003182', 'unknown', 'need to check'), ('003973', 500, 'en'), ('000261', 500, 'en'), ('001131', 'unknown', 'need to check'), ('013667', 500, 'en'), ('009762', 'unknown', 'need to check'), ('004503', 500, 'en'), ('010585', 500, 'en'), ('000529', 500, 'en'), ('001833', 500, 'en'), ('010498', 500, 'en'), ('000316', 500, 'en'), ('004222', 500, 'en'), ('001957', 500, 'en'), ('009284', 500, 'en'), ('010505', 500, 'en'), ('001649', 'unknown', 'need to check'), ('001698', 500, 'en'), ('001831', 500, 'en'), ('005234', 'unknown', 'need to check'), ('000087', 500, 'en'), ('000090', 500, 'en'), ('000862', 500, 'en'), ('001841', 500, 'en'), ('005312', 500, 'en'), ('002984', 500, 'en'), ('001630', 500, 'en'), ('011422', 500, 'en'), ('001825', 500, 'en'), ('000863', 500, 'en'), ('013573', 'unknown', 'need to check'), ('001536', 500, 'en'), ('010809', 500, 'en'), ('000250', 500, 'en'), ('000639', 'unknown', 'need to check'), ('000866', 'unknown', 'need to check'), ('002962', 500, 'en'), ('003464', 500, 'en'), ('013683', 'unknown', 'need to check'), ('004053', 500, 'en'), ('009273', 500, 'en'), ('010372', 'unknown', 'need to check'), ('000442', 500, 'en'), ('001986', 'unknown', 'need to check'), ('009315', 500, 'en'), ('001625', 500, 'en'), ('001779', 500, 'en'), ('000186', 'unknown', 'need to check'), ('001569', 'unknown', 'need to check'), ('011011', 500, 'en'), ('010601', 'unknown', 'need to check'), ('000884', 500, 'en'), ('004927', 500, 'en'), ('001253', 500, 'en'), ('005083', 500, 'en'), ('010955', 500, 'en'), ('000465', 500, 'en'), ('000257', 500, 'en'), ('004615', 500, 'en'), ('007437', 500, 'en'), ('011134', 500, 'en'), ('010743', 500, 'en'), ('001586', 'unknown', 'need to check'), ('002345', 500, 'en'), ('000379', 500, 'en'), ('000963', 500, 'en'), ('001497', 500, 'en'), ('000883', 500, 'en'), ('001770', 500, 'en'), ('001608', 500, 'en'), ('002253', 500, 'en'), ('010584', 'unknown', 'need to check'), ('000464', 500, 'en'), ('011182', 500, 'en'), ('001647', 500, 'en'), ('000125', 500, 'en'), ('000444', 500, 'en'), ('000382', 500, 'en'), ('006948', 'unknown', 'need to check'), ('001808', 'unknown', 'need to check'), ('000967', 500, 'en'), ('000206', 500, 'en'), ('004852', 500, 'en'), ('001251', 500, 'en'), ('009370', 500, 'en'), ('010305', 500, 'en'), ('005571', 500, 'en'), ('004275', 500, 'en'), ('003893', 500, 'en'), ('010216', 'unknown', 'need to check')]

In [ ]:
BASE = '/content/drive/MyDrive/KB/Data/'
lang_type = 'en'
for id in tqdm(same_jo_mst):
    url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&ID={id}'
    response_en = requests.get(url)
    json_data_en = xmltodict.parse(response_en.text)
    with open(BASE+lang_type+f"/en_{id}.json", 'w') as f:
        json.dump(json_data_en, indent=4, ensure_ascii=False, fp=f)

100%|██████████| 1670/1670 [37:19<00:00,  1.34s/it]


In [ ]:
lang_type = 'kr'
for id in tqdm(same_jo_mst):
    url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&ID={id}'
    response_kr = requests.get(url)
    json_data_kr = xmltodict.parse(response_kr.text)
    with open(BASE+lang_type+f"/kr_{id}.json", 'w') as f:
        json.dump(json_data_kr, indent=4, ensure_ascii=False, fp=f)

100%|██████████| 1670/1670 [59:55<00:00,  2.15s/it]


In [ ]:
# @title
def check_jo(json_data_kr, json_data_en):
    tmp = list()
    curr = -1
    for t in json_data_kr['법령']['조문']['조문단위']:
        if t['조문여부'] == '조문':
            if '조문가지번호' in t:
                tmp.append(f"{int(curr)}-{t['조문번호']}-{t['조문가지번호']}")#({t['조문시행일자'] if '조문시행일자' in t else ''})")
            else:
                tmp.append(f"{int(curr)}-{t['조문번호']}")
        if t['조문여부'] == '전문':
            curr = t['조문번호']
    tmp2 = list()
    curr = -1
    for t in json_data_en['Law']['JoSection']['Jo']:
        if t['joYn'] == 'Y':
            if 'joBrNo' in t and int(t['joBrNo']) != 0:
                tmp2.append(f"{int(curr)}-{int(t['joNo'])}-{int(t['joBrNo'])}")
            else:
                tmp2.append(f"{int(curr)}-{int(t['joNo'])}")
        if t['joYn'] == 'N':
            curr = t['joNo']
    return tmp, tmp2 # kr, en

In [ ]:
os.chdir('/content/drive/MyDrive/KB')

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

df = pd.DataFrame(columns=["이름", "한국어", "영어"])
err = []

for file in tqdm(os.listdir('Data/kr/')):
    _, a = os.path.split(f'Data/kr/{file}')
    filename, form = os.path.splitext(a)
    filename = filename[3:]
    with open(f'Data/kr/{file}', 'r') as f:
        json_data_kr = json.load(f)
    with open(f'Data/en/en_{filename}.json', 'r') as f:
        json_data_en = json.load(f)
    law_name = json_data_kr['법령']['기본정보']['법령명_한글']
    cnt_kr, cnt_en = check_jo(json_data_kr, json_data_en)
    if cnt_kr == cnt_en:
        for kr, en in zip(json_data_kr['법령']['조문']['조문단위'], json_data_en['Law']['JoSection']['Jo']):
            cnt = 1
            # print(kr['조문내용'])
            if '항' in kr:
                kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
                cnt += len(kr_h)
                for kr_hang in kr_h:
                    if '항내용' not in kr_hang:
                        cnt -= 1
                    else:
                        pass
                        # print("\t", kr_hang['항내용'])
                    if '호' in kr_hang:
                        kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
                        cnt += len(kr_hos)
                        for kr_ho in kr_hos:
                            if '호내용' not in kr_ho:
                                cnt -= 1
                            else:
                                pass
                                # print("\t\t", kr_ho['호내용'])
                            if '목' in kr_ho:
                                cnt += len(kr_ho['목'])
                                for kr_mok in kr_ho['목']:
                                    if '목내용' not in kr_mok:
                                        cnt -= 1
                                    else:
                                        # print("\t\t\t", kr_mok['목내용'])
                                        pass
            en_val = len(en['joCts'].replace(') (1)', ')\n(1)').split('\n'))
            en_val2 = sum([1 if t.strip().startswith('[') else 0 for t in en['joCts'].split('\n')])
            en_val -= en_val2
            if cnt == en_val:
                en_law = en['joCts'].replace(') (1)', ')\n  (1)').split('\n')
                cnt = 0
                if '항' in kr:
                    kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
                    # print(kr['조문내용'], en_law[cnt])
                    # kr_data.append(kr['조문내용'])
                    # en_data.append(en_law[cnt])
                    row = {
                        "이름": law_name,
                        "한국어": kr['조문내용'],
                        "영어": en_law[cnt]
                    }
                    df = df.append(row, ignore_index=True)
                    cnt += 1
                    for kr_hang in kr_h:
                        if '항내용' not in kr_hang:
                            pass
                        else:
                            pass
                            # print(kr_hang['항내용'], en_law[cnt])
                            # kr_data.append(kr_hang['항내용'])
                            # en_data.append(en_law[cnt])
                            row = {
                                "이름": law_name,
                                "한국어": kr_hang['항내용'],
                                "영어": en_law[cnt]
                            }
                            df = df.append(row, ignore_index=True)
                            cnt += 1
                        if '호' in kr_hang:
                            kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
                            for kr_ho in kr_hos:
                                if '호내용' not in kr_ho:
                                    pass
                                else:
                                    # print("\t\t", kr_ho['호내용'], en_law[cnt])
                                    # kr_data.append(kr_ho['호내용'])
                                    # en_data.append(en_law[cnt])
                                    row = {
                                        "이름": law_name,
                                        "한국어": kr_ho['호내용'],
                                        "영어": en_law[cnt]
                                    }
                                    df = df.append(row, ignore_index=True)
                                    cnt += 1
                                    pass

                                if '목' in kr_ho:
                                    for kr_mok in kr_ho['목']:
                                        if '목내용' not in kr_mok:
                                            pass
                                        else:
                                            # print("\t\t\t", kr_mok['목내용'], en_law[cnt])
                                            # kr_data.append(kr_mok['목내용'])
                                            # en_data.append(en_law[cnt])
                                            row = {
                                                "이름": law_name,
                                                "한국어": kr_mok['목내용'],
                                                "영어": en_law[cnt]
                                            }
                                            df = df.append(row, ignore_index=True)
                                            cnt += 1
                                            pass

In [ ]:
df.to_csv("/content/drive/MyDrive/KB/df.csv", index=False)

In [50]:
df = pd.read_csv('/content/drive/MyDrive/KB/df.csv')
df['category'] = ''

In [51]:
## data category 추가

import re
pattern = re.compile(r'(\d+\.\s*)(.*)(\s*\[시행)')

for file in tqdm(os.listdir('Data/type/')):
    name = file[:-4]
    with open(f'Data/type/{file}', 'r') as f:
        lines = f.readlines()
    lines = set([pattern.search(x).group(2).strip() for x in lines if pattern.search(x)])
    mask = df['이름'].isin(lines)
    df.loc[mask, 'category'] = name

100%|██████████| 44/44 [00:21<00:00,  2.03it/s]


In [56]:
df.to_csv("/content/drive/MyDrive/KB/df_type.csv", index=False)